# Notebook Walkthrough of Database Creation + Processing

As seen in the image below, this notebook just walks through the assumptions, filters and processing steps taken to create the SQL database used in the main analysis notebook.

In specific, this notebook walks us through the following steps:
1. Data Sources
2. Website APIs
3. Key Matching
4. Data Storage


<p align="center" width="10%">
    <img width="50%" src="images/tech_stack.png"> 
</p>

<div style="text-align: left; font-size: 16px; color: #224CA0 ;">
    <strong> Code Architecture Note: </strong>
</div>

The following cells employ 1 class for API interfacing, requests and SQL table curation. Huge thank you to vishaalagartha and swar for their github repos that already provided the webscraping framework needed to easily extract all the needed data.

To see the API commands used, please see the 'Database_Operations.py' file.

With that being said, the first step before is of course importing the helper class.

In [ ]:
#Import Helper functions for databases extraction (steps )
import sqlite3
import Database_Operations

# Step 1 - Data Sources
The next step is specify the desired seasons of interest (in this study, that is the past 10 years going back to the 2011-2012 season). We then initialize the API_Handlers class to perform the web scraping and queries needed to agglomerate the data.

<br>
<div style="text-align: left; font-size: 22px; color: #7a1019 ;">
    <strong> Side Note: </strong>
</div>

I'm not sure if I am just missing something but why does nba.com not provide player salaries!? The whole reason SQL db key matching was needed is because I had to pool multiple sources to get player salaries and stats. The worst part is the naming of players was inconsistent even on a given website. To give an idea of the mismatches, here are a few examples that I was able to systematically correct with string matching techniques:

| Mismatch | BBall Ref | NBA.com |
|-----------------|-----------------|-----------------|
| Special Characters | Jokić | Jokic |
| Name Suffix | Jaren Jackson Jr. | Jaren Jackson |
| Suffix Annotation | Gary Trent Jr. | Gary Trent Jr |
| Abbreviations | JJ Barea | Jose Juan Barea Mora |

Yes regular expression could probably solve a lot of the problems but using a much faster/easier method, I was able to get salary/player matching error down to <strong> 4.5% </strong> which is pretty good in my opinion. I analyze this error metric a little more in the main notebook by the way.

In [ ]:
#Define years of interest
valid_years = [str(1999 + i) for i in range(25)]

#Initialize API_Handler
API_Handler = Database_functions.API_Handlers(valid_years)

# Step 2 - String Matching to Link Multiple Data Sources (Website APIs)
The next step is string match player names. I created an automated function that lets you say y/n to a player match but a lot of it was just automated based on if a string match was above or below 80% scoring and the player salary. A lot of the mismatches come from players who basically earned 0$ on non-guaranteed contracts, 10 days, G league call ups, etc. Its pretty easy to just remove these datapoints and replace them with generic placeholder values since the overall influence is negligble on team building from a salary standpoint.

In [ ]:
#Get matched player salaries from hoopshype
ref_players,df_sal = API_Handler.import_hoopshype_data()

#Make sure all stats are matched based on a salary cutoff and assumptions mentioned above.
player_data = API_Handler.player_matching(ref_players,
                                          api_players,
                                          df_sal,
                                          plot=True,
                                          cutoff=2500000,
                                          pickler=True)

# Step 3 - Data Storage 
The last step before we get into the fun analysis part is creating a sql db and its tables. The key matching structure is visualized below

<p align="center" width="10%">
    <img width="70%" src="images/sql_keys.png"> 
</p>

In [ ]:
#create SQL db 
API_Handler.generate_sqlite3_db('player_stats','NBA_stats_ghub.db')

#create supporting tables
API_Handler.generate_player_stats_table('Player_stats', player_data)
#Other tables dont need the df since the class handles the explicit generation of the data already
API_Handler.generate_salary_table('Player_Salaries')
API_Handler.generate_team_table('Team_stats')

#create a temporary table to varify real team salaries vs. calculated salaries
API_Handler.generate_team_caps_table('team_caps')

Now lets move on to the interesting part... the data analysis to see what we can extract from simple data